In [2]:
from google.colab import drive
drive.mount('/content/drive/')
import os

root_path = "/content/drive/My Drive/강우예측AI"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
!cp -r /content/drive/My\ Drive/강우예측AI .

In [4]:
import zipfile

In [5]:
try:
    with zipfile.ZipFile("강우예측AI/train.zip") as zf:
        zf.extractall()
        print("uncompress success")

except:
  pass

uncompress success


In [6]:
try:
    with zipfile.ZipFile("강우예측AI/test.zip") as zf:
        zf.extractall()
        print("uncompress success")

except:
  pass

uncompress success


In [7]:
!cp 강우예측AI/sample_submission.csv .

In [8]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, BatchNormalization, concatenate, Input
from tensorflow.keras import Model
import imgaug
import warnings
warnings.filterwarnings("ignore")

In [9]:
# train data 생성
train_files = glob.glob('/content/train/*.npy')
len(train_files)
train_files = np.array(train_files)

In [10]:
# train 데이터 생성
len(train_files)

62735

In [11]:
def trainGenerator():
    for file in tr_file:
        dataset = np.load(file)
        target= dataset[:,:,-1].reshape(120,120,1)        
        remove_minus = np.where(target < 0, 0, target)
        feature = dataset[:,:,:4]



        yield (feature, remove_minus)

def valGenerator():
    for file in val_file:
        dataset = np.load(file)
        target= dataset[:,:,-1].reshape(120,120,1)        
        remove_minus = np.where(target < 0, 0, target)
        feature = dataset[:,:,:4]


        yield (feature, remove_minus)

In [12]:
# 최초 베이스라인
def base_model(input_layer, start_neurons):
    
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    pool1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D((2, 2))(pool1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    pool2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2, 2))(pool2)

    convm = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
    
    return output_layer

In [13]:
import gc
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=42)
for fold, (train, val) in enumerate(kf.split(train_files)):
  #print(type(val))
  val_file = train_files[val]
  tr_file = train_files[train]
  

  train_dataset = tf.data.Dataset.from_generator(trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([120,120,4]),tf.TensorShape([120,120,1])))
  train_dataset = train_dataset.shuffle(256)

  train_dataset = train_dataset.batch(128).prefetch(1)

  val_dataset = tf.data.Dataset.from_generator(valGenerator, (tf.float32, tf.float32), (tf.TensorShape([120,120,4]),tf.TensorShape([120,120,1])))
  val_dataset = val_dataset.batch(128).prefetch(1)

  

  input_layer = Input((120, 120, 4))
  output_layer = base_model(input_layer,64)

  model = Model(input_layer, output_layer)
  adam = tf.keras.optimizers.Adam()
  model.compile(optimizer=adam, loss='mae')


  callbacks = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=2, verbose=0, mode='min',
    min_delta=0.0001, cooldown=0, min_lr=0
  )

  sv = tf.keras.callbacks.ModelCheckpoint(
        os.path.join(root_path,f'{fold}-rain.h5'), monitor='val_loss', verbose=0, save_best_only=True,
        save_weights_only=True, mode='min', save_freq='epoch')

  model.fit(train_dataset, epochs = 1, verbose=1, validation_data=val_dataset, callbacks=[callbacks, sv])

  del model
  gc.collect()

      2/Unknown - 0s 160ms/step - loss: 11.5687WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1135s vs `on_train_batch_end` time: 0.2056s). Check your callbacks.
393/393 [==============================] - 154s 393ms/step - loss: 6.0194 - val_loss: 3.5382
      2/Unknown - 0s 162ms/step - loss: 13.2906WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1171s vs `on_train_batch_end` time: 0.2065s). Check your callbacks.
393/393 [==============================] - 155s 395ms/step - loss: 5.8701 - val_loss: 5.6270
      2/Unknown - 0s 159ms/step - loss: 14.5268WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1125s vs `on_train_batch_end` time: 0.2051s). Check your callbacks.
393/393 [==============================] - 142s 361ms/step - loss: 5.9309 - val_loss: 3.5728
      2/Unknown - 0s 161ms/step - loss: 14.0107WARNING:

In [14]:
import gc
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
def loss(final_X_answers, final_X_pred):
  pred = final_X_pred
  answer = final_X_answers
  mae = mean_absolute_error(answer, pred)
  print(mae)

In [15]:
input_layer = Input((120, 120, 4))
output_layer = base_model(input_layer,64)
model = Model(input_layer, output_layer)

In [16]:
test_files = sorted(glob.glob('test*.npy'))
X_test = []
for file in tqdm(test_files, desc = 'test'):
    data = np.load(file)
    X_test.append(data)

X_test = np.array(X_test)

test: 100%|██████████| 2674/2674 [00:04<00:00, 592.99it/s]


In [17]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=42)
total_val = []
total_answer = []
test_preds = []

for fold, (train, val) in enumerate(kf.split(train_files)):

  val_file = train_files[val]

  X_val = []
  X_answers = []
  
  for file in tqdm(val_file, desc='val'):
    data = np.load(file)
    X_answer = data[:,:,-1].reshape(14400)
    X_input = data[:,:,:4]
  
    X_val.append(X_input)
    X_answers.append(X_answer)
  X_val = np.array(X_val)
  X_answers = np.array(X_answers)
  model.load_weights(os.path.join(root_path,f'{fold}-rain.h5'))

  val_pred = model.predict(X_val)
  test_pred = model.predict(X_test)

  test_pred = test_pred.reshape(-1,14400)
  final_X_val = val_pred.reshape(-1,14400)
  total_val.append(final_X_val)
  total_answer.append(X_answers)
  test_preds.append(test_pred)
  
  del val_pred, X_val, X_answers, final_X_val
  gc.collect()

val: 100%|██████████| 12547/12547 [00:30<00:00, 408.42it/s]


In [18]:
final_preds = (test_preds[0]+test_preds[1]+test_preds[2]+test_preds[3]+test_preds[4])/5
sub = pd.read_csv("sample_submission.csv")
sub.iloc[:,1:] = final_preds.astype(int)
sub.to_csv("submission.csv", index=False)
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>